In [1]:
%load_ext autoreload

In [2]:
import sys
import os
import time
import numpy as np
from functools import partial
import itertools
import mcubes
import visualizations_utils as viz_utils
import iris_utils #TODO remove
from iris_plant_visualizer import IrisPlantVisualizer

In [3]:
#pydrake imports
from pydrake.common import FindResourceOrThrow
from pydrake.multibody.parsing import LoadModelDirectives, Parser, ProcessModelDirectives
from pydrake.multibody.plant import MultibodyPlant, AddMultibodyPlantSceneGraph
from pydrake.systems.framework import DiagramBuilder
from pydrake.all import InverseKinematics, RevoluteJoint
import pydrake.symbolic as sym
from pydrake.all import MathematicalProgram, RationalForwardKinematics
from pydrake.geometry.optimization import IrisOptionsRationalSpace, IrisInRationalConfigurationSpace, HPolyhedron, Hyperellipsoid

import meshcat

# Build plant

In [4]:

builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)
parser = Parser(plant)
parser.package_map().Add( "wsg_50_description", os.path.dirname(FindResourceOrThrow(
            "drake/manipulation/models/wsg_50_description/package.xml")))

simple_collision = True
use_gripper = False
if simple_collision and use_gripper:
    directives_file = FindResourceOrThrow("drake/sos_iris_certifier/planar_iiwa_simple_collision_welded_gripper.yaml")
elif not simple_collision and use_gripper:
    directives_file = FindResourceOrThrow("drake/sos_iris_certifier/planar_iiwa_dense_collision_welded_gripper.yaml") 
elif simple_collision and not use_gripper:
    directives_file = FindResourceOrThrow("drake/sos_iris_certifier/planar_iiwa_simple_collision_no_gripper.yaml")
elif not simple_collision and not use_gripper:
    directives_file = FindResourceOrThrow("drake/sos_iris_certifier/planar_iiwa_dense_collision_no_gripper.yaml") 
directives = LoadModelDirectives(directives_file)
models = ProcessModelDirectives(directives, plant, parser)

q0 = [-0.2, -1.2, 1.6]
index = 0
for joint_index in plant.GetJointIndices(models[0].model_instance):
    joint = plant.get_mutable_joint(joint_index)
    if isinstance(joint, RevoluteJoint):
        joint.set_default_angle(q0[index])
        index += 1

plant.Finalize()
# visualizer = ConnectMeshcatVisualizer(builder, scene_graph, zmq_url=zmq_url, 
#                                       delete_prefix_on_load=False)

# diagram = builder.Build()
# visualizer.load()


# Setup meshcat visualization

In [5]:
do_viz = True
visualizer = IrisPlantVisualizer(plant, builder, scene_graph)
diagram = visualizer.diagram

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7004/static/
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7005/static/
Connecting to meshcat-server at zmq_url=tcp://127.0.0.1:6004...
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7004/static/
Connected to meshcat-server.


# Run SNOPT IRIS in rational space

In [6]:
iris_options = IrisOptionsRationalSpace()
iris_options.require_sample_point_is_contained = True
iris_options.iteration_limit = 5
iris_options.configuration_space_margin = 1e-4
iris_options.termination_threshold = -1
iris_options.relative_termination_threshold = 0.05
iris_options.enable_ibex = False
iris_options.certify_region_with_sos_during_generation = False
iris_options.certify_region_with_sos_after_generation = False
# uncomment to test if non-default is working
# currently throwing std::bad_alloc if we try to set q_star
iris_options.q_star = 0.5 * np.zeros(3)
rational_forward_kinematics = RationalForwardKinematics(plant)


#seed points specified in q space
seed_points_q = np.array([
                        [0.0, -2.016, 1.975], # in tight
                        [-1, -2, 0.5],        # neutral pose
                        [0.3, -0.8, 0.5],     # above shelf
                        [0.25, -1.6, -0.25],  # in shelf 1
                        [0.07, -1.8, -0.2],   # leaving shelf 1
                        [-0.1, -2, -0.3]      # out of shelf 1
                        ]) 
seed_points_t = np.array([rational_forward_kinematics.ComputeTValue(s, iris_options.q_star) for s in seed_points_q])

t_lower = rational_forward_kinematics.ComputeTValue(plant.GetPositionLowerLimits(),iris_options.q_star)
t_upper = rational_forward_kinematics.ComputeTValue(plant.GetPositionUpperLimits(),iris_options.q_star)
P_joint_limits = HPolyhedron.MakeBox(t_lower, t_upper)

In [7]:
regions = []
context = diagram.CreateDefaultContext()
for i, s in enumerate(seed_points_q):
    plant.SetPositions(plant.GetMyMutableContextFromRoot(context), s)
    if i > 0:
        starting_hpolyhedron = regions[i-1]
        r = IrisInRationalConfigurationSpace (plant, plant.GetMyContextFromRoot(context),
                                              iris_options, starting_hpolyhedron)
    else:
        r = IrisInRationalConfigurationSpace(plant, plant.GetMyContextFromRoot(context), iris_options)
    regions.append(r)
    print(f'Completed region: {i+1}/{len(seed_points_q)}')

Completed region: 1/6
Completed region: 2/6
Completed region: 3/6
Completed region: 4/6
Completed region: 5/6
Completed region: 6/6


In [8]:
# plot regions and collision constraint
if do_viz:
    visualizer.plot_regions(regions)
    visualizer.plot_seedpoints(seed_points_t)
    visualizer.visualize_collision_constraint(N = 50)

# Begin Certification

In [9]:
import pydrake.multibody.rational_forward_kinematics as rational_forward_kinematics
from pydrake.solvers import mathematicalprogram as mp
cspace_free_region = rational_forward_kinematics.CspaceFreeRegion(diagram, plant, scene_graph,
                                   rational_forward_kinematics.SeparatingPlaneOrder.kAffine,
                                   rational_forward_kinematics.CspaceRegionType.kGenericPolytope)

In [11]:
editted_regions = []
filtered_collision_pairs = set()

binary_search_options = rational_forward_kinematics.BinarySearchOption()
binary_search_options.epsilon_max = 10
binary_search_options.epsilon_min = 0
binary_search_options.max_iters = 5
binary_search_options.search_d = False

bilinear_alternation_option = rational_forward_kinematics.BilinearAlternationOption()
bilinear_alternation_option.max_iters = 2
bilinear_alternation_option.lagrangian_backoff_scale = 1e-3
bilinear_alternation_option.polytope_backoff_scale = 1e-5

solver_options = mp.SolverOptions()
for r in regions: 
    try:
        d_feasible = \
            cspace_free_region.CspacePolytopeBinarySearch(
                iris_options.q_star, filtered_collision_pairs, r.A(), r.b(),
                binary_search_options, solver_options)
        C_final, d_final, P_final, q_final = \
            cspace_free_region.CspacePolytopeBilinearAlternation(
                iris_options.q_star, filtered_collision_pairs, r.A(), d_feasible,
                bilinear_alternation_option, solver_options)
        C_final = np.vstack([C_final, P_joint_limits.A()])
        d_final = np.concatenate([d_final, P_joint_limits.b()])
        editted_regions.append(HPolyhedron(C_final, d_final))
    except Exception as e:
        print(e)
print(len(editted_regions))

[2022-01-20 21:06:46.192] [console] [info] Solver time 0.034253835678100586
[2022-01-20 21:06:46.390] [console] [info] Solver time 0.08045315742492676
[2022-01-20 21:06:46.531] [console] [info] Solver time 0.03246903419494629
[2022-01-20 21:06:46.532] [console] [info] epsilon=5.0 is infeasible
[2022-01-20 21:06:46.701] [console] [info] Solver time 0.048013925552368164
[2022-01-20 21:06:46.703] [console] [info] epsilon=2.5 is infeasible
[2022-01-20 21:06:46.870] [console] [info] Solver time 0.04705691337585449
[2022-01-20 21:06:46.871] [console] [info] epsilon=1.25 is infeasible
[2022-01-20 21:06:47.043] [console] [info] Solver time 0.05081892013549805
[2022-01-20 21:06:47.044] [console] [info] epsilon=0.625 is infeasible
[2022-01-20 21:06:47.222] [console] [info] Solver time 0.05555891990661621
[2022-01-20 21:06:47.223] [console] [info] epsilon=0.3125 is infeasible
[2022-01-20 21:06:47.565] [console] [info] Iter: 0, max(log(det(P)))=0.5891707686996777, solver_time 0.24892687797546387
[

Backoff fails.


[2022-01-20 21:06:48.366] [console] [info] Solver time 0.03340911865234375
[2022-01-20 21:06:48.367] [console] [info] epsilon=5.0 is infeasible
[2022-01-20 21:06:48.512] [console] [info] Solver time 0.03960704803466797
[2022-01-20 21:06:48.513] [console] [info] epsilon=2.5 is infeasible
[2022-01-20 21:06:48.683] [console] [info] Solver time 0.05614280700683594
[2022-01-20 21:06:48.685] [console] [info] epsilon=1.25 is infeasible
[2022-01-20 21:06:48.831] [console] [info] Solver time 0.03965401649475098
[2022-01-20 21:06:48.832] [console] [info] epsilon=0.625 is infeasible
[2022-01-20 21:06:48.992] [console] [info] Solver time 0.051867008209228516
[2022-01-20 21:06:48.993] [console] [info] epsilon=0.3125 is infeasible
[2022-01-20 21:06:49.226] [console] [info] Iter: 0, max(log(det(P)))=1.0831367496630486, solver_time 0.1461019515991211
[2022-01-20 21:06:49.344] [console] [info] backoff with log(det(P)) 1.0820972346529034, solver time 0.1046450138092041
[2022-01-20 21:06:49.345] [console

binary search: the initial epsilon 0.0 is infeasible
binary search: the initial epsilon 0.0 is infeasible


[2022-01-20 21:06:51.217] [console] [info] Solver time 0.09026384353637695
[2022-01-20 21:06:51.396] [console] [info] Solver time 0.03409099578857422
[2022-01-20 21:06:51.591] [console] [info] Solver time 0.08016109466552734


binary search: the initial epsilon 0.0 is infeasible
binary search: the initial epsilon 0.0 is infeasible
1


In [12]:
if do_viz:
    visualizer.plot_regions(editted_regions, region_suffix = 'editted')

In [13]:
editted_regions[0].A()

array([[ 9.50832304e-01,  1.98057519e-07, -2.87680174e-08],
       [-1.27476529e-08,  1.00000000e+00,  1.26723372e-08],
       [ 3.70189840e-08, -3.63289952e-08,  6.23159874e-09],
       [-9.50832394e-01, -9.20429921e-09, -7.19887532e-08],
       [-1.04015361e-07, -1.00000000e+00,  7.26697881e-08],
       [ 8.18179956e-08, -5.49746351e-08,  2.61331400e-09],
       [ 9.47849330e-01, -2.36037111e-01,  1.65937522e-03],
       [ 1.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  1.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00],
       [-1.00000000e+00, -0.00000000e+00, -0.00000000e+00],
       [-0.00000000e+00, -1.00000000e+00, -0.00000000e+00],
       [-0.00000000e+00, -0.00000000e+00, -1.00000000e+00]])

In [14]:
editted_regions[0].b()

array([0.92168414, 2.12004113, 0.54831819, 2.16132594, 2.68725155,
       0.67460231, 0.26956007, 1.73205081, 1.73205081, 1.73205081,
       1.73205081, 1.73205081, 1.73205081])